In [1]:
import xgboost

In [2]:
import pandas as pd

In [3]:
dataset=pd.read_csv("insurance_pre.csv")

In [4]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [6]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [7]:
dataset=dataset.astype(int)

In [8]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27,0,16884,0,1
1,18,33,1,1725,1,0
2,28,33,3,4449,1,0
3,33,22,0,21984,1,0
4,32,28,0,3866,1,0
...,...,...,...,...,...,...
1333,50,30,3,10600,1,0
1334,18,31,0,2205,0,0
1335,18,36,0,1629,0,0
1336,21,25,0,2007,0,0


In [9]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [10]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [11]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27,0,0,1
1,18,33,1,1,0
2,28,33,3,1,0
3,33,22,0,1,0
4,32,28,0,1,0
...,...,...,...,...,...
1333,50,30,3,1,0
1334,18,31,0,0,0
1335,18,36,0,0,0
1336,21,25,0,0,0


In [12]:
dependent=dataset[['charges']]

In [13]:
dependent

,charges
0,16884
1,1725
2,4449
3,21984
4,3866
...,...
1333,10600
1334,2205
1335,1629
1336,2007


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=0.3,random_state=0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
from sklearn.ensemble import GradientBoostingRegressor

In [18]:
param_grid={'loss':['squared_error', 'absolute_error', 'huber', 'quantile'],'criterion':['friedman_mse', 'squared_error'],
           'max_features':['sqrt', 'log2']}

In [19]:
grid=GridSearchCV(GradientBoostingRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)

In [20]:
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\sreet\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'squared_error'],
                         'loss': ['squared_error', 'absolute_error', 'huber',
                                  'quantile'],
                         'max_features': ['sqrt', 'log2']},
             verbose=3)

In [21]:
re=grid.cv_results_

In [22]:
print("The R_score value for best parameter{}:".format(grid.best_params_))

The R_score value for best parameter{'criterion': 'friedman_mse', 'loss': 'huber', 'max_features': 'log2'}:


In [23]:
table=pd.DataFrame.from_dict(re)

In [24]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_loss,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.109195,0.003164,0.003275,0.002551,friedman_mse,squared_error,sqrt,"{'criterion': 'friedman_mse', 'loss': 'squared...",0.879529,0.820802,0.832940,0.848350,0.794517,0.835228,0.028304,8
1,0.109935,0.003483,0.003002,0.002451,friedman_mse,squared_error,log2,"{'criterion': 'friedman_mse', 'loss': 'squared...",0.881589,0.826142,0.829226,0.846343,0.798700,0.836400,0.027272,6
2,0.427087,0.008978,0.002215,0.002521,friedman_mse,absolute_error,sqrt,"{'criterion': 'friedman_mse', 'loss': 'absolut...",0.875008,0.830794,0.810750,0.831403,0.785150,0.826621,0.029507,9
3,0.424085,0.020094,0.008175,0.009062,friedman_mse,absolute_error,log2,"{'criterion': 'friedman_mse', 'loss': 'absolut...",0.865488,0.828910,0.819805,0.804448,0.747052,0.813141,0.038670,11
4,0.495127,0.022241,0.002326,0.002157,friedman_mse,huber,sqrt,"{'criterion': 'friedman_mse', 'loss': 'huber',...",0.880520,0.824632,0.836794,0.847330,0.796262,0.837108,0.027616,4
5,0.546583,0.014228,0.002189,0.002464,friedman_mse,huber,log2,"{'criterion': 'friedman_mse', 'loss': 'huber',...",0.884736,0.831522,0.838533,0.847687,0.797727,0.840041,0.028005,1
6,0.419169,0.026979,0.003158,0.002595,friedman_mse,quantile,sqrt,"{'criterion': 'friedman_mse', 'loss': 'quantil...",0.493698,0.524446,0.465350,0.564385,0.722009,0.553978,0.090226,16
7,0.391834,0.006212,0.004010,0.002005,friedman_mse,quantile,log2,"{'criterion': 'friedman_mse', 'loss': 'quantil...",0.508724,0.502340,0.524890,0.557519,0.701516,0.558998,0.073776,15
8,0.109739,0.002351,0.001326,0.001102,squared_error,squared_error,sqrt,"{'criterion': 'squared_error', 'loss': 'square...",0.876819,0.820881,0.839963,0.849066,0.798582,0.837062,0.026365,5
9,0.107859,0.002076,0.003754,0.002324,squared_error,squared_error,log2,"{'criterion': 'squared_error', 'loss': 'square...",0.879937,0.821194,0.830570,0.844959,0.803165,0.835965,0.025836,7


In [25]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=float(input("Sex_Male 0 or 1:"))
smoker_yes_input=float(input("Smoker_Yes 0 or 1:"))

Age:24
BMI:60
Children:3
Sex_Male 0 or 1:1
Smoker_Yes 0 or 1:0


In [26]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])

In [27]:
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[27544.81703991]
